In [1]:
from datetime import datetime
def str2time(s):
    ans = None
    if s.startswith('今天'):
        ans = datetime.today()
        t = datetime.strptime(s.split(' ')[1], '%H:%M')
        ans = ans.replace(hour = t.hour, minute = t.minute)
        return ans
    try:
        ans = datetime.strptime(s, '%m月%d日 %H:%M')
        ans = ans.replace(year = datetime.today().year)
        return ans
    except:
        try:
            ans = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
            return ans
        except:
            return None

In [7]:
def get_features(user):
    vector = {
        'reputation'            : 0,
        'mention'               : 0,
        'hashtag'               : 0,
        'url'                   : 0,
        'text_similarity'       : 0,
        'time_interval_mean'    : 0,
        'time_interval_var'     : 0,
        'post_num'              : 0,
        'active_day_ratio'      : 0,
        'follow_ratio'          : 0,
        'figure_RRT'            : 0,
        'average_comm'          : 0,
        'followee_num'          : 0,
        'figure_at_every'       : 0,
        'day_interval_variance' : 0,
        'late_night_times'      : 0,
        'figure_at_sigle'       : 0,
        'figure_at'             : 0
    }
    
    # 声望：关注他的人与他关注的人之比
    try:
        vector['reputation'] = float(user['followers_num']) / float(user['following_num'])
    except:
        return dict()
    
    import jieba
    # '@'次数和'#'次数：
    vector['mention'] = 0
    vector['hashtag'] = 0
    vector['url']     = 0
    posts   = []
    time_intervals    = []
    prev_time         = None
    for post in user['weibo']:
        if '@' in post['content']:
            vector['mention'] += 1
        if '#' in post['content']:
            vector['hashtag'] += 1
        if 'http://' in post['content']:
            vector['url'] += 1
        posts.append(list(jieba.cut(post['content'])))
        
        curr_time = str2time(' '.join(post['time'].split()[:2]))
        if prev_time != None:
            time_intervals.append((prev_time - curr_time).seconds)
        prev_time = curr_time
    
    import numpy as np
    vector['time_interval_mean'] = np.mean(time_intervals)
    vector['time_interval_var'] = np.var(time_intervals)
    
    vector['post_num'] = int(user['weibos_num'])
    
    return vector

    from gensim.models import word2vec
    # posts之间的文本相似度
    words = word2vec.Text8Corpus('text.txt') #暂时还没有语料
    model = word2vec.Word2Vec(words, size=200)
    sum = .0
    num = 0
    for i in range(len(posts)):
        for j in range(i + 1, len(posts)):
            sum += model.similarity(posts[i], posts[j])
            num += 1
    vector['test_similarity'] = sum / num
    
    return vector

In [8]:
import os
import re
import json

def get_label(text, id):
    p = re.compile('%s.+?(yes|no)' % id)
    return re.findall(p, text)[0]
    

path = './spider/weibo/'
files = os.listdir(path)
text = open('./data/spammer_order.csv', 'r').read()

vectors = []
for file in files:
    user = json.load(open(os.path.join(path, file), 'r'))
    if int(user['weibos_num']) <= 0:
        continue
    id = file.split('.')[0]
    label = get_label(text, id)
    vector 
    vectors.append([] + list(get_features(user).values()))

In [9]:
for v in vectors:
    print(v)

['1200860993', 1.3046122655854029, 14, 10, 7, 0, 29546.125, 959912634.56770837, 9025, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['1215990923', 0.2948801036941024, 0, 0, 36, 0, 25443.367346938776, 287128768.47730112, 1087, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['1495710180', 0.5993690851735016, 5, 6, 4, 0, 14777.340425531915, 459719669.62879133, 10201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['1568793054', 0.26046511627906976, 5, 19, 3, 0, 30074.918367346938, 1103328695.2178259, 1380, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['1796759141', 0.20909886264216973, 1, 18, 0, 0, 29392.65306122449, 919768501.12453151, 2017, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['2065072254', 0.3516483516483517, 2, 7, 6, 0, 17197.367346938776, 719538209.1711787, 18528, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['2102882751', 1.2007648183556405, 11, 10, 17, 0, 15431.081632653062, 793681915.70762181, 3837, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['2161663171', 0.24193548387096775, 14, 24, 1, 0, 37201.333333333336, 860043438.22222221, 7561, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['2320739702